# Unit tests con TensorFlow

In [2]:
import sys
import numpy as np
import tensorflow as tf

In [3]:
session = tf.Session()

In [4]:
data_dir = "../../datasets/MNIST_data/"
mnist = tf.keras.datasets.mnist
(train_xdata, train_labels), (test_xdata, test_labels) = mnist.load_data()

In [5]:
train_xdata = train_xdata/255.0
test_xdata = test_xdata/255.0

In [15]:
batch_size = 100
learning_rate = 0.005
evaluation_size = 100
image_width = train_xdata[0].shape[0]
image_height = train_xdata[0].shape[1]
target_size = max(train_labels) + 1
num_channels = 1
generations = 100
eval_every = 5
conv1_features = 25
conv2_features = 50
max_pool_size1 = 2
max_pool_size2 = 2
fully_connected_size1 = 100
dropout_prob = 0.75

In [16]:
x_input_shape = (batch_size, image_width, image_height, num_channels)
x_input = tf.placeholder(shape=x_input_shape, dtype=tf.float32)
y_target = tf.placeholder(shape=(batch_size), dtype=tf.int32)

eval_input_shape = (evaluation_size, image_width, image_height, num_channels)
eval_input = tf.placeholder(shape=eval_input_shape, dtype=tf.float32)
eval_target = tf.placeholder(shape=(evaluation_size), dtype=tf.float32)

dropout = tf.placeholder(dtype=tf.float32, shape=())

In [17]:
conv1_weight = tf.Variable(tf.truncated_normal(shape=[4,4,num_channels,conv1_features], stddev=0.1, 
                                               dtype=tf.float32))
conv1_bias = tf.Variable(tf.zeros(shape=[conv1_features], dtype=tf.float32))

conv2_weight = tf.Variable(tf.truncated_normal(shape=[4,4,conv1_features,conv2_features], stddev=0.1, 
                                               dtype=tf.float32))
conv2_bias = tf.Variable(tf.zeros(shape=[conv2_features], dtype=tf.float32))

In [21]:
result_width = image_width//(max_pool_size1+max_pool_size2)
result_height = image_height//(max_pool_size1+max_pool_size2)
full1_input_size = result_height*result_width*conv2_features

full1_weight = tf.Variable(tf.truncated_normal(shape=[full1_input_size, fully_connected_size1], stddev=0.1, 
                                               dtype=tf.float32))
full1_bias = tf.Variable(tf.zeros(shape=[fully_connected_size1], dtype=tf.float32))

output_weight = tf.Variable(tf.truncated_normal(shape=[fully_connected_size1, target_size], stddev=0.1, 
                                               dtype=tf.float32))
output_bias = tf.Variable(tf.zeros(shape=[target_size]))

In [26]:
def cnn(input_data):
    conv1 = tf.nn.conv2d(input=input_data, filter=conv1_weight, strides=[1,1,1,1], padding="SAME")
    relu1 = tf.nn.relu(tf.nn.bias_add(value=conv1, bias=conv1_bias))
    maxpool1 = tf.nn.max_pool(value=relu1, ksize=[1,max_pool_size1,max_pool_size1,1], 
                              strides=[1,max_pool_size1,max_pool_size1,1], padding="SAME")
    
    conv2 = tf.nn.conv2d(input=maxpool1, filter=conv2_weight, strides=[1,1,1,1], padding="SAME")
    relu2 = tf.nn.relu(tf.nn.bias_add(value=conv2, bias=conv2_bias))
    maxpool2 = tf.nn.max_pool(value=relu2, ksize=[1,max_pool_size2,max_pool_size2,1], 
                              strides=[1,max_pool_size2,max_pool_size2,1], padding="SAME")
    
    final_conv_shape = maxpool2.get_shape().as_list()
    final_shape = final_conv_shape[1] * final_conv_shape[2] * final_conv_shape[3]
    flat_output = tf.reshape(maxpool2, [final_conv_shape[0], final_shape])
    
    fully_connected1 = tf.nn.relu(tf.add(tf.matmul(flat_output, full1_weight), full1_bias))
    fully_connected2 = tf.add(tf.matmul(fully_connected1, output_weight), output_bias)
    
    return tf.nn.dropout(fully_connected2, keep_prob=dropout_prob)

model_output = cnn(x_input)
test_output = cnn(eval_input)

In [27]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_output, labels=y_target))

In [28]:
prediction = tf.nn.softmax(model_output)
test_prediction = tf.nn.softmax(test_output)

In [48]:
def get_accuracy(logits, labels):
    predictions = np.argmax(logits, axis=1)
    corrects = np.sum(np.equal(predictions, labels))
    return 100*corrects/predictions.shape[0]

In [30]:
train = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss)

In [31]:
init = tf.global_variables_initializer()
session.run(init)

### Pruebas unitarias

In [32]:
class DropOutTest(tf.test.TestCase):
    def dropout_greater_than(self):
        with self.test_session():
            self.assertGreater(dropout.eval(), 25)

In [33]:
class AccuracyTest(tf.test.TestCase):
    def accuracy_exact_test(self):
        with self.test_session():
            test_preds = [[0.9, 0.1], [0.01, 0.99]]
            test_targets = [0,1]
            test_acc = get_accuracy(test_preds, test_targets)
            self.assertEqual(test_acc.eval(), 100.0)

In [34]:
class ShapeTest(tf.test.TestCase):
    def output_shape_test(self):
        with self.test_session():
            numpy_array = np.zeros([batch_size, target_size])
            self.assertShapeEqual(numpy_array, model_output)

In [42]:
def main(argv):
    train_loss = []
    train_acc = []
    test_acc = []
    
    for i in range(generations):
        rand_idx = np.random.choice(len(train_xdata), size=batch_size)
        rand_x = np.expand_dims(train_xdata[rand_idx], 3)
        rand_y = train_labels[rand_idx]
        train_dict = {x_input: rand_x, y_target: rand_y, dropout: dropout_prob}
        
        session.run(train, feed_dict=train_dict)
        
        temp_train_loss, temp_train_preds = session.run([loss, prediction], feed_dict=train_dict)
        temp_train_acc = get_accuracy(temp_train_preds, rand_y)
        
        if (i+1)%eval_every == 0:
            eval_idx = np.random.choice(len(test_xdata), size=evaluation_size)
            eval_x = np.expand_dims(test_xdata[eval_idx], 3)
            eval_y = test_labels[eval_idx]
            test_dict = {eval_input: eval_x, eval_target: eval_y, dropout: 1.0}
            
            test_preds = session.run(test_prediction, feed_dict=test_dict)
            temp_test_acc = get_accuracy(test_preds, eval_y)
            
            train_loss.append(temp_train_loss)
            train_acc.append(temp_train_acc)
            test_acc.append(temp_test_acc)
            
            acc_and_loss = [(i+1), temp_train_loss, temp_train_acc, temp_test_acc]
            acc_and_loss = [np.round(x,2) for x in acc_and_loss]
            print("Step #{} ==> Train loss: {}, Train acc: {}, Test acc: {}".format(*acc_and_loss))

In [49]:
if __name__ == "__main__":
    cmd_args = sys.argv
    if len(cmd_args) > 1 and cmd_args[1] == "test":
        tf.test.main(argv=cmd_args[1:])
    else:
        tf.app.run(main=None, argv=cmd_args)

Step #5 ==> Train loss: 2.069999933242798, Train acc: 21.0, Test acc: 20.0
Step #10 ==> Train loss: 1.9199999570846558, Train acc: 34.0, Test acc: 35.0
Step #15 ==> Train loss: 1.7000000476837158, Train acc: 49.0, Test acc: 40.0
Step #20 ==> Train loss: 1.5099999904632568, Train acc: 53.0, Test acc: 52.0
Step #25 ==> Train loss: 1.1799999475479126, Train acc: 63.0, Test acc: 58.0
Step #30 ==> Train loss: 1.1699999570846558, Train acc: 61.0, Test acc: 63.0
Step #35 ==> Train loss: 1.090000033378601, Train acc: 63.0, Test acc: 68.0
Step #40 ==> Train loss: 0.9700000286102295, Train acc: 62.0, Test acc: 67.0
Step #45 ==> Train loss: 0.9200000166893005, Train acc: 66.0, Test acc: 64.0
Step #50 ==> Train loss: 0.800000011920929, Train acc: 70.0, Test acc: 64.0
Step #55 ==> Train loss: 0.8199999928474426, Train acc: 74.0, Test acc: 73.0
Step #60 ==> Train loss: 0.9700000286102295, Train acc: 69.0, Test acc: 77.0
Step #65 ==> Train loss: 0.6899999976158142, Train acc: 76.0, Test acc: 65.0
Ste

SystemExit: 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
